# Variational Auto Encoder from *Auto-Encoding Variational Bayes*

In [23]:
import keras
import numpy as np
import h5py

In [47]:
def vae(Xtrain, nndim = 200, latentdim = int(2), samplestd = 1.0,
        epochs = int(10), batch_size = int(100)):
    inputdim = Xtrain.shape[1]
    """define an auto-encoder following AEVB"""
    
    if inputdim <= nndim or inputdim <= latentdim:
        print("overspecified model")
        return -1
    
    if batch_size > Xtrain.shape[0]:
        print("too large batch size")
        return -1
    
    if epochs < 1:
        print("must have at least one epoch")
        return -1
    
    X = keras.layers.Input(shape = (inputdim,))
    Xinv = keras.layers.Dense(units = inputdim, activation = "sigmoid")
    Q = keras.layers.Dense(object = X, units = nndim, activation = "relu")
    Qinv = keras.layers.Dense(units = nndim, activation = "relu")
    
    Zm = keras.layers.Dense(object = Q, units = latentdim, activation = "linear")
    Zv = keras.layers.Dense(object = Q, units = latentdim, activation = "linear")
    
    def lam(det):
        """lambda for keras"""
        Zm = det[:, range(latentdim)]
        Zv = det[:, range(latentdim+1,2*latentdim)]
        noise = keras.backend.random_normal(shape = keras.shape(Zm)[0],
                                           mean = 0,
                                           stddev=samplestd)
        return Zm + noise * keras.backend.sqrt(keras.backend.exp(Zv))
    
    Z = keras.layers.Lambda(keras.layers.Concatenate([Zm, Zv]), lam)
    
    Zgen = keras.layers.Input(shape = latentdim)
    
    Qout = Qinv(Z)
    Xout = Xinv(Qout)
    
    Qgen = Qinv(Zgen)
    Xgen = Xinv(Qgen)
    
    autoencoder = keras.Model(X, Xout)
    encoder = keras.Model(X, Zm)
    decoder = keras.Model(Zgen, Xgen)
    
    l= 1.0
    
    def AEVBloss(X, Xout):
        """Loss from Auto-Encoding Variational Bayes page 5"""
        decterm = (inputdim/l) * keras.losses.binary_crossentropy(X, Xout)
        KLterm = -0.5 * keras.backend.mean(1 + Zv - 
                                           keras.backend.square(Zm) - 
                                          keras.backend.exp(Zv),
                                          axis = -1)
        return KLterm + decterm
    
    
    autoencoder.compile(optimizer = "rmsprop", loss = AEVBloss)
    
    autoencoder.fit(x = Xtrain, y = Xtrain, epochs = epochs, batch_size = batch_size)
    
    return([autoencoder, encoder, decoder])

In [33]:
from keras.datasets import mnist
mnist.load_data()

11493376/11490434 [==============================] - 6s 0us/step


((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [34]:
(Xtrain, ytrain), (Xtest, ytest) = mnist.load_data()

In [36]:
Xtrain.shape

(60000, 28, 28)

In [38]:
Xtrain = Xtrain.reshape(60000, 784)

In [41]:
Xtrain = Xtrain / 255

In [48]:
output = vae(Xtrain, nndim = 200, latentdim = int(2), samplestd = 1.0,
        epochs = int(10), batch_size = int(100))

TypeError: ('Keyword argument not understood:', 'object')